#GreenHawk data prepration
In this notebook we have the code we used to covert the annotation format,  create the train/val/test splits and create the subset for the initial seed model.

Please note that not all parts can be followed through as some file handeling was done manually.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = 'train.csv'
df = pd.read_csv(file_name)


In [ ]:
df.describe()

,class,xmax,xmin,ymax,ymin
count,19950.000000,19950.000000,19950.000000,19950.000000,19950.000000
mean,4.055388,572.813634,362.418647,314.202506,201.514937
std,2.502491,264.445257,264.246281,102.133871,85.536614
min,0.000000,30.000000,-397.000000,20.000000,-150.000000
25%,3.000000,357.000000,147.000000,242.000000,149.000000
50%,3.000000,567.000000,350.000000,296.000000,195.000000
75%,4.000000,789.000000,581.000000,372.000000,251.000000
max,10.000000,1384.000000,935.000000,698.000000,500.000000


In [ ]:
df.head()

,class,image_path,name,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0


#Converting xml annotation to YOLO annotation

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    # from xmax	xmin	ymax	ymin
    xmin = max(bbox[1],0)
    ymin = max(bbox[3],0)
    xmax = min(bbox[0],w)
    ymax = min(bbox[2],h)
    x_center = ((xmax + xmin) / 2) / w
    y_center = ((ymax + ymin) / 2) / h
    width = (xmax - xmin) / w
    height = (ymax - ymin) / h
    return [x_center, y_center, width, height]


In [ ]:
training_file_names = df.image_path.unique()    


In [ ]:
for tf_name in training_file_names:
  # print(tf_name)
  objects_in_tf_name = df[df.image_path == tf_name]
  # print(objects_in_tf_name)

  with open('/content/labels/'+tf_name[:-4]+'.txt', 'a') as f:
    for i in range(len(objects_in_tf_name)):
      row = objects_in_tf_name.iloc[i]
      box = xml_to_yolo_bbox(row.values[3:],960,540)
      f.write(f"{int(row.values[0])} {' '.join(f'{x:.6f}' for x in box)}\n")
    f.close()
    

In [ ]:
!zip -r /content/labels.zip /content/labels


Streaming output truncated to the last 5000 lines.
  adding: content/labels/fddaa811aad3aa8e2f34728ee0584ff2.txt (deflated 39%)
  adding: content/labels/f4ae1829272047019bb15d011a29d19c.txt (deflated 13%)
  adding: content/labels/5c03cc523d55120c1a69d953748ebd2f.txt (deflated 21%)
  adding: content/labels/70028aae96348de8e82110530d4cd784.txt (deflated 28%)
  adding: content/labels/5edb1bd2719ad0f87da40bff1d62c8bf.txt (deflated 41%)
  adding: content/labels/572c40a23d2630f475350de7f5ff543a.txt (deflated 29%)
  adding: content/labels/98ed3e03cef662a362c53489f50847a2.txt (deflated 29%)
  adding: content/labels/68e1ef58613955524f2de95f68d38798.txt (deflated 29%)
  adding: content/labels/e37f84be1fc6fbbf4eef0e920b7bc670.txt (deflated 13%)
  adding: content/labels/a8ec1f73cefe2c4058dded1438728cae.txt (deflated 13%)
  adding: content/labels/18576210d16f4854faebc828dd74cf59.txt (deflated 16%)
  adding: content/labels/b9fb2f5ea9b84f42005804878d64038e.txt (deflated 5%)
  adding: content/labels/1

In [ ]:
from PIL import Image, ImageDraw


def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax]


def draw_image(img, bboxes):
    draw = ImageDraw.Draw(img)
    for bbox in bboxes:
        draw.rectangle(bbox, outline="red", width=2)
    img.save("example.jpg")
    img.show()



In [ ]:
fname = '5ba59eaae491b93fc28b7c60bd3aaaf7.jpg'
# /content/images/0a39d1bcea61bb0a20eba63f76c66eb3.jpg
image_filename = "/content/"+fname
label_filename = "/content/"+fname[:-3]+"txt"
bboxes = []

img = Image.open(image_filename)

with open(label_filename, 'r', encoding='utf8') as f:
    for line in f:
        data = line.strip().split(' ')
        bbox = [float(x) for x in data[1:]]
        print(data[0])
        print(data[1:])
        bboxes.append(yolo_to_xml_bbox(bbox, img.width, img.height))

draw_image(img, bboxes)


4
['0.276042', '0.500000', '0.552083', '1.000000']
4
['0.663542', '0.567593', '0.672917', '0.864815']


#Creating train/val/test splits

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)
from pathlib import Path    

Mounted at /content/drive


In [ ]:
import random
import shutil
import os
trainNames = []
vSmartathonNames = []
train_path_images = '/content/drive/MyDrive/GreenHawk/Dataset/TrainF/images/'
train_path_labels = '/content/drive/MyDrive/GreenHawk/Dataset/TrainF/labels/'
vSmartathon_path_images = '/content/drive/MyDrive/GreenHawk/Dataset/ValF/images/'
vSmartathon_path_labels = '/content/drive/MyDrive/GreenHawk/Dataset/ValF/labels/'
os.makedirs(train_path_images) if not os.path.exists(train_path_images) else None
os.makedirs(train_path_labels) if not os.path.exists(train_path_labels) else None
os.makedirs(vSmartathon_path_images) if not os.path.exists(vSmartathon_path_images) else None
os.makedirs(vSmartathon_path_labels) if not os.path.exists(vSmartathon_path_labels) else None

for tf_name in training_file_names:
  if random.choice([0,1,2,3,4,5,6,7,8,9]) != 0:
    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/images/'+ tf_name
    dst_path = train_path_images + tf_name
    shutil.copy(src_path, dst_path)

    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/labels/'+ tf_name[:-4]+'.txt'
    dst_path = train_path_labels + tf_name[:-4]+'.txt'
    shutil.copy(src_path, dst_path)
    trainNames.append(tf_name[:-4])
  else:
    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/images/'+ tf_name
    dst_path = vSmartathon_path_images + tf_name
    shutil.copy(src_path, dst_path)

    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/labels/'+ tf_name[:-4]+'.txt'
    dst_path = vSmartathon_path_labels + tf_name[:-4]+'.txt'
    shutil.copy(src_path, dst_path)
    vSmartathonNames.append(tf_name[:-4])

In [ ]:
dftest = pd.read_csv('test.csv')
testing_file_names = dftest.image_path.unique()    


In [ ]:
tSmartathon_path_images = '/content/drive/MyDrive/GreenHawk/Dataset/TestF/images/'
os.makedirs(tSmartathon_path_images) if not os.path.exists(tSmartathon_path_images) else None

for tf_name in testing_file_names:
    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/images/'+ tf_name
    dst_path = tSmartathon_path_images + tf_name
    shutil.copy(src_path, dst_path)


#Adding random 300 images to build the initial model for the active learning cycle

In [ ]:
randSmartathon_path_images = '/content/drive/MyDrive/GreenHawk/Dataset/WRP/images/'
os.makedirs(randSmartathon_path_images) if not os.path.exists(randSmartathon_path_images) else None
randSmartathon_path_labels = '/content/drive/MyDrive/GreenHawk/Dataset/WRP/labels/'
os.makedirs(randSmartathon_path_labels) if not os.path.exists(randSmartathon_path_labels) else None

count = 0
for tf_name in trainNames:
  if random.choice([0,1,2,3,4,5]) == 0:
    count = count + 1
    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/images/'+ tf_name +'.jpg'
    dst_path = randSmartathon_path_images + tf_name
    shutil.copy(src_path, dst_path)

    src_path = '/content/drive/MyDrive/GreenHawk/Dataset/All/labels/'+ tf_name+'.txt'
    dst_path = randSmartathon_path_labels + tf_name+'.txt'
    shutil.copy(src_path, dst_path)
  if count == 300:
    break;
